## Import and Setup

In [81]:
from numbers_parser import Document
import pandas as pd
import plotly.graph_objects as go

## Loading data from Numbers file

In [82]:
doc = Document("/Users/guillaume/Library/Mobile Documents/com~apple~Numbers/Documents/Job Applications.numbers")
sheets = doc.sheets()
tables = sheets[0].tables()
data = tables[0].rows(values_only=True)
df = pd.DataFrame(data, columns=["Company Name", "Job Title", "Category", "Place", "Date", "Answer", "Interview", "Mail", "Pwd", "Note"])
df.drop(index=df.index[0], axis=0, inplace=True)

## Preping the Sankey data 

In [83]:
INIT=0
TBA=1
REJECTED=2
INTERVIEW=3
OFFER=4
TDOWN=5

source=[]
target=[]
value=[]

answers = df["Answer"].value_counts()

### Adding the first layer for the "No Answer" applications

In [84]:
source.append(INIT)
target.append(TBA)
value.append(len(df[(df["Interview"]=="No") & (df["Answer"]=="Tba")]))

### Adding a second layer for the interviews

In [85]:
source.append(INIT)
target.append(INTERVIEW)
value.append(df["Interview"].value_counts()["Yes"])

### Adding the "Offer" and "Turned Down" layers

In [86]:
source.append(INTERVIEW)
target.append(OFFER)
value.append(len(df[(df["Interview"]=="Yes") & (df["Answer"]=="Offer")]))

source.append(INTERVIEW)
target.append(TDOWN)
value.append(len(df[(df["Interview"]=="Yes") & (df["Answer"]=="Turned Down")]))

### Addind the directly rejected layer

In [87]:
source.append(INIT)
target.append(REJECTED)
value.append(len(df[(df["Interview"]=="No") & (df["Answer"]=="Rejected")]))

### Adding the layer of rejection after interviewing

In [88]:
source.append(INTERVIEW)
target.append(REJECTED)
value.append(len(df[(df["Interview"]=="Yes") & (df["Answer"]=="Rejected")]))

### Adding the last layer for the interviews without answer

In [89]:
source.append(INTERVIEW)
target.append(TBA)
value.append(len(df[(df["Interview"]=="Yes") & (df["Answer"]=="Tba")]))

## Sankey Visualisation

In [93]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label = ["Applied", "No Answer", "Rejected", "Interviews", "Offer", "Turned Down"],
      color = ["black", "orange", "red", "blue", "green", "brown"]
    ),
    link = dict(
      source = source,
      target = target,
      value = value,
  ))])

fig.update_layout(title_text="Job Hunt", font_size=10)
fig.show()